In [1]:
import numpy as np
import mlflow

from mlflow.models import infer_signature
import torch
from torch import nn
import pandas as  pd

In [5]:
model_name = "pytorch_model.pt"
model_version = 1

model = mlflow.pyfunc.load_model(model_uri=f"..mlartifacts/:models/{model_name}")

print(model.metadata)

OSError: No such file or directory: '..mlartifacts/:models/pytorch_model.pt'

In [ ]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

net = nn.Linear(10, 1)
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)

X = torch.randn(100, 10)
y = torch.randn(100, 1)

print(X.shape, y.shape)

epochs = 5
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = net(X)

    loss = loss_function(outputs, y)
    loss.backward()

    optimizer.step()

with mlflow.start_run() as run:
    signature = infer_signature(X.numpy(), net(X).detach().numpy())
    model_info = mlflow.pytorch.log_model(
        pytorch_model = net, 
        artifact_path = "pytorch model", 
        signature=signature, 
        input_example=X.numpy(),
        registered_model_name="pytorch_model"
        )

    pytorch_pyfunc = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)

    X_test = torch.randn(20, 10).numpy()
    predictions = pytorch_pyfunc.predict(X_test)
    print(predictions)

    
    eval_data = pd.DataFrame(X.numpy())
    eval_data = pd.DataFrame(y.numpy())
    print(eval_data)
    eval_data.columns = ["label"]
    eval_data["predictions"] = net(X).detach().numpy()
    print(eval_data.shape)
    
    
    results = mlflow.evaluate(
        data=eval_data,
        model_type="regressor",
        targets= "label",
        predictions="predictions",
        evaluators = ["default"]
    )

print(f"metrics:\n{results.metrics}")
print(f"artifacts:\n{results.artifacts}")

In [ ]:
import mlflow.pyfunc
from mlflow import MlflowClient

run_id = "e389609f9f1b44678ea7fea020453f94"
model_artifact_path = "pytorch model"

model = mlflow.pyfunc.load_model(model_uri=f"runs:/{run_id}/{model_artifact_path}")

print(model.metadata)

# OR

model_name = "best"
model_version = 1

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

print(model.metadata)

# OR

client = MlflowClient()
client.set_registered_model_alias(name = model_name, alias = "staging", version = "1")

model_name = "pytorch_model"
model_alias = "staging"

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}@{model_alias}")

print(model.metadata)